# everyrow SDK Basic Usage

This notebook demonstrates all the core operations in the everyrow SDK:

1. **Screen** - Filter rows based on criteria that need judgment
2. **Rank** - Score rows by qualitative factors
3. **Dedupe** - Deduplicate when fuzzy matching fails
4. **Merge** - Join tables when keys don't match exactly
5. **Derive** - Add computed columns (no AI needed)
6. **Single Agent** - Web research on a single input
7. **Agent Map** - Web research on every row of a dataframe

Get an API key at [everyrow.io/api-key](https://everyrow.io/api-key) to run this notebook.

In [1]:
import os
from textwrap import dedent

from dotenv import load_dotenv
from pandas import DataFrame
from pydantic import BaseModel, Field

load_dotenv()

True

## 1. Screen

Filter rows based on criteria you can't put in a WHERE clause. This example performs vendor risk assessment - evaluating security track records and financial stability requires judgment, not pattern matching.

In [2]:
from everyrow.ops import screen


class VendorRiskAssessment(BaseModel):
    approved: bool
    risk_level: str  # "low", "medium", "high"
    security_concerns: str
    financial_stability_notes: str
    recommendation: str


vendors = DataFrame(
    [
        {"company": "Okta", "category": "Identity Management", "website": "okta.com"},
        {"company": "LastPass", "category": "Password Management", "website": "lastpass.com"},
        {"company": "Snowflake", "category": "Data Warehouse", "website": "snowflake.com"},
        {"company": "Cloudflare", "category": "CDN & Security", "website": "cloudflare.com"},
        {"company": "MongoDB", "category": "Database", "website": "mongodb.com"},
    ]
)

print("Input vendors:")
print(vendors.to_string())

Input vendors:
      company             category         website
0        Okta  Identity Management        okta.com
1    LastPass  Password Management    lastpass.com
2   Snowflake       Data Warehouse   snowflake.com
3  Cloudflare       CDN & Security  cloudflare.com
4     MongoDB             Database     mongodb.com


In [ ]:
screen_result = await screen(
    task=dedent("""
        Perform vendor risk assessment for each company. Research and evaluate:

        1. Security track record: Have they had any significant data breaches or security
        incidents in the past 3 years? How did they respond?

        2. Financial stability: Are there signs of financial distress (major layoffs,
        funding difficulties, declining revenue)?

        3. Overall recommendation: Based on your research, should we proceed with
        this vendor for enterprise use?

        Only approve vendors with low or medium risk and no unresolved critical security incidents.
    """),
    input=vendors,
    response_model=VendorRiskAssessment,
)

print("Vendor Risk Assessment Results:")
print(screen_result.data.to_string())

## 2. Rank

Score rows by things you can't put in a database field. This example ranks AI research organizations by leadership citation counts - information that requires researching each org's leaders and their publications.

In [4]:
from everyrow.ops import rank


class ContributionRanking(BaseModel):
    contribution_score: int = Field(description="Total citation count")
    most_significant_contribution: str = Field(
        description="Single most important paper authored by a firm leader"
    )


ai_research_orgs = DataFrame(
    [
        {"organization": "OpenAI", "type": "Private lab", "founded": 2015},
        {"organization": "Google DeepMind", "type": "Corporate lab", "founded": 2010},
        {"organization": "Anthropic", "type": "Private lab", "founded": 2021},
        {"organization": "Meta FAIR", "type": "Corporate lab", "founded": 2013},
        {"organization": "Microsoft Research", "type": "Corporate lab", "founded": 1991},
        {"organization": "Stanford HAI", "type": "Academic", "founded": 2019},
    ]
)

print("AI Research Organizations:")
print(ai_research_orgs.to_string())

AI Research Organizations:
         organization           type  founded
0              OpenAI    Private lab     2015
1     Google DeepMind  Corporate lab     2010
2           Anthropic    Private lab     2021
3           Meta FAIR  Corporate lab     2013
4  Microsoft Research  Corporate lab     1991
5        Stanford HAI       Academic     2019


In [5]:
rank_task = dedent("""
    Research the total citation count of all leaders of the given AI research organization.

    A leader is defined as a C-Suite or founder of the company.
    Citation count should count all major publications. Top ten by each person is sufficient.
""")

# Basic ranking with a single score field
rank_result = await rank(
    task=rank_task,
    input=ai_research_orgs,
    field_name="contribution_score",
    ascending_order=False,
)

print("AI Research Organization Rankings:")
print(rank_result.data.to_string())

AI Research Organization Rankings:
         organization           type  founded                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [6]:
# Ranking with a custom response model for additional context
detailed_rank_result = await rank(
    task=rank_task + "\n\nAlso include their single most significant contribution.",
    input=ai_research_orgs,
    field_name="contribution_score",
    response_model=ContributionRanking,
    ascending_order=False,
)

print("Detailed Rankings with Context:")
print(detailed_rank_result.data.to_string())

Detailed Rankings with Context:
         organization           type  founded  contribution_score                                                                                                                                                   most_significant_contribution                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

## 3. Dedupe

Deduplicate when fuzzy matching falls short. This example deduplicates academic papers where the same paper may appear with different identifiers (arXiv ID vs DOI), different title formats, or as preprint vs published versions.

In [7]:
from everyrow.ops import dedupe

papers = DataFrame(
    [
        {
            "title": "Attention Is All You Need",
            "authors": "Vaswani et al.",
            "venue": "NeurIPS 2017",
            "identifier": "10.5555/3295222.3295349",
        },
        {
            "title": "Attention Is All You Need",
            "authors": "Vaswani, Shazeer, Parmar et al.",
            "venue": "arXiv",
            "identifier": "1706.03762",
        },
        {
            "title": "BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding",
            "authors": "Devlin et al.",
            "venue": "NAACL 2019",
            "identifier": "10.18653/v1/N19-1423",
        },
        {
            "title": "BERT: Pre-training of Deep Bidirectional Transformers",
            "authors": "Devlin, Chang, Lee, Toutanova",
            "venue": "arXiv",
            "identifier": "1810.04805",
        },
        {
            "title": "Language Models are Few-Shot Learners",
            "authors": "Brown et al.",
            "venue": "NeurIPS 2020",
            "identifier": "GPT-3",
        },
        {
            "title": "GPT-3: Language Models are Few-Shot Learners",
            "authors": "Brown, Mann, Ryder et al.",
            "venue": "arXiv",
            "identifier": "2005.14165",
        },
        {
            "title": "LLaMA: Open and Efficient Foundation Language Models",
            "authors": "Touvron et al.",
            "venue": "arXiv",
            "identifier": "2302.13971",
        },
        {
            "title": "Llama 2: Open Foundation and Fine-Tuned Chat Models",
            "authors": "Touvron et al.",
            "venue": "arXiv",
            "identifier": "2307.09288",
        },
    ]
)

print(f"Input papers ({len(papers)} rows):")
print(papers.to_string())

Input papers (8 rows):
                                                                              title                          authors         venue               identifier
0                                                         Attention Is All You Need                   Vaswani et al.  NeurIPS 2017  10.5555/3295222.3295349
1                                                         Attention Is All You Need  Vaswani, Shazeer, Parmar et al.         arXiv               1706.03762
2  BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding                    Devlin et al.    NAACL 2019     10.18653/v1/N19-1423
3                             BERT: Pre-training of Deep Bidirectional Transformers    Devlin, Chang, Lee, Toutanova         arXiv               1810.04805
4                                             Language Models are Few-Shot Learners                     Brown et al.  NeurIPS 2020                    GPT-3
5                                      GP

In [8]:
dedupe_result = await dedupe(
    input=papers,
    equivalence_relation=dedent("""
        Two entries are duplicates if they represent the same research work, which requires
        verifying through research:

        - An arXiv preprint and its published conference/journal version are duplicates
        - Papers with slightly different titles but same core contribution are duplicates
        - Different author list formats (et al. vs full list) don't matter
        - Papers with different identifiers (arXiv ID vs DOI) may still be duplicates

        However, genuinely different papers (e.g., LLaMA 1 vs LLaMA 2) are NOT duplicates,
        even if authors and topics overlap. Research each paper to determine if they
        report the same findings or are distinct works.
    "),
)

print("Deduplicated Paper List:")
print(dedupe_result.data.to_string())
print(f"\nOriginal entries: {len(papers)}")
print(f"Unique papers: {len(dedupe_result.data)}")
print(f"Duplicates removed: {len(papers) - len(dedupe_result.data)}")

_IncompleteInputError: incomplete input (95882388.py, line 3)

## 4. Merge

Join two tables when the keys don't match exactly. This example merges clinical trial data with pharmaceutical company information - the challenge is that trial sponsors are often subsidiaries or use abbreviated names (e.g., "MSD" instead of "Merck").

In [ ]:
from everyrow.ops import merge

clinical_trials = DataFrame(
    [
        {
            "trial_id": "NCT05432109",
            "sponsor": "Genentech",
            "indication": "Non-small cell lung cancer",
            "phase": "Phase 3",
        },
        {
            "trial_id": "NCT05891234",
            "sponsor": "Janssen Pharmaceuticals",
            "indication": "Multiple myeloma",
            "phase": "Phase 2",
        },
        {
            "trial_id": "NCT05567890",
            "sponsor": "MSD",
            "indication": "Melanoma",
            "phase": "Phase 3",
        },
        {
            "trial_id": "NCT05234567",
            "sponsor": "AbbVie Inc",
            "indication": "Rheumatoid arthritis",
            "phase": "Phase 3",
        },
        {
            "trial_id": "NCT05678901",
            "sponsor": "BMS",
            "indication": "Acute myeloid leukemia",
            "phase": "Phase 2",
        },
    ]
)

pharma_companies = DataFrame(
    [
        {
            "company": "Roche Holding AG",
            "hq_country": "Switzerland",
            "2024_revenue_billions": 58.7,
        },
        {
            "company": "Johnson & Johnson",
            "hq_country": "United States",
            "2024_revenue_billions": 85.2,
        },
        {
            "company": "Merck & Co.",
            "hq_country": "United States",
            "2024_revenue_billions": 60.1,
        },
        {
            "company": "AbbVie",
            "hq_country": "United States",
            "2024_revenue_billions": 56.3,
        },
        {
            "company": "Bristol-Myers Squibb",
            "hq_country": "United States",
            "2024_revenue_billions": 45.0,
        },
    ]
)

print("Clinical Trials:")
print(clinical_trials.to_string())
print("\nPharma Companies:")
print(pharma_companies.to_string())

In [ ]:
merge_result = await merge(
    task=dedent("""
        Merge clinical trial data with parent pharmaceutical company information.

        The sponsor names in the trials table are often subsidiaries or abbreviations:
        - Research which parent company owns each trial sponsor
        - Match trials to their parent company's financial data

        For example, Genentech is a subsidiary of Roche, Janssen is part of J&J,
        MSD is Merck's name outside the US, BMS is Bristol-Myers Squibb.
    """),
    left_table=clinical_trials,
    right_table=pharma_companies,
    merge_on_left="sponsor",
    merge_on_right="company",
)

print("Clinical Trials with Parent Company Data:")
print(merge_result.data.to_string())

## 5. Derive

Add computed columns using pandas expressions - no AI agents needed. This is useful for simple calculated fields before or after other operations.

In [ ]:
from everyrow.ops import derive

orders = DataFrame(
    [
        {"product": "Widget", "price": 10.00, "quantity": 5},
        {"product": "Gadget", "price": 25.50, "quantity": 3},
        {"product": "Gizmo", "price": 7.25, "quantity": 10},
    ]
)

print("Input data:")
print(orders.to_string())

In [ ]:
derive_result = await derive(
    input=orders,
    expressions={
        "total": "price * quantity",
    },
)

print("With derived 'total' column:")
print(derive_result.data.to_string())

## 6. Single Agent

Run web research on a single input. Agents can generate tabular data or answer questions based on research. This example first generates a competitor dataset, then analyzes it for market gaps.

In [ ]:
from everyrow.ops import single_agent


class Competitor(BaseModel):
    company: str = Field(description="Company name")
    pricing_tier: str = Field(description="Pricing model, e.g. 'Freemium, $10-50/user/mo'")
    target_market: str = Field(description="Primary customer segment")
    key_features: str = Field(description="Top 3 features or differentiators")


# Step 1: Generate a dataset of competitors
print("Step 1: Research competitors")
competitors = await single_agent(
    task="Find the top 10 competitors in the B2B expense management software market",
    response_model=Competitor,
    return_table=True,
)
print(competitors.data.to_string())

In [ ]:
# Step 2: Distill insights from the dataset
print("Step 2: Identify market gaps")
insights = await single_agent(
    task="""
        What gaps exist in the B2B expense management software market
        that these competitors aren't addressing?
    """,
    input=competitors,
)
print(insights.data.answer)

## 7. Agent Map

Run web research on every row of a dataframe. This example researches financial information for tech companies - information that requires looking up each company individually.

In [ ]:
from everyrow.ops import agent_map


class CompanyFinancials(BaseModel):
    annual_revenue_usd: int = Field(description="Most recent annual revenue in USD")
    employee_count: int = Field(description="Current number of employees")
    founded_year: int = Field(description="Year the company was founded")


companies = DataFrame(
    [
        {"company": "Stripe", "industry": "Payments"},
        {"company": "Databricks", "industry": "Data & AI"},
        {"company": "Canva", "industry": "Design"},
        {"company": "Figma", "industry": "Design"},
        {"company": "Notion", "industry": "Productivity"},
    ]
)

print("Companies to research:")
print(companies.to_string())

In [ ]:
# Basic usage with default response
basic_result = await agent_map(
    task="Find the company's most recent annual revenue in USD",
    input=companies,
)

print("Basic Results:")
print(basic_result.data.to_string())

In [ ]:
# Structured output with a response model
structured_result = await agent_map(
    task=dedent("""
        Research the company's financials. Find:
        1. Their most recent annual revenue (in USD)
        2. Current employee count
        3. Year founded

        If the company is a subsidiary, report figures for the subsidiary
        specifically, not the parent company.
    """),
    input=companies,
    response_model=CompanyFinancials,
)

print("Structured Results:")
print(structured_result.data.to_string())